In [1]:
# Step 1: Import necessary libraries
import pandas as pd  # For data manipulation
import numpy as np  # For numerical operations
import re  # For regular expression operations
import sqlite3       # For interacting with SQLite databases

In [2]:
data = pd.read_csv('[CONFIDENTIAL] AI symptom picker data (Agnos candidate assignment) - ai_symptom_picker - [CONFIDENTIAL] AI symptom picker data (Agnos candidate assignment) - ai_symptom_picker.csv') # Path to the CSV file

In [3]:
data.head()  # Display the first few rows of the DataFrame

,gender,age,summary,search_term
0,male,28,"{""diseases"": [], ""procedures"": [], ""no_symptom...","มีเสมหะ, ไอ"
1,male,27,"{""diseases"": [], ""procedures"": [], ""no_symptom...","ไอ, น้ำมูกไหล"
2,female,26,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ปวดท้อง
3,male,42,"{""diseases"": [], ""procedures"": [], ""no_symptom...",น้ำมูกไหล
4,female,40,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ตาแห้ง


In [4]:
data.drop(['gender', 'age'], axis=1, inplace=True)

In [5]:
# Step 2 : Extract - extrct only important words
def extract_answers(summary):
    """
    Extracts the 'answers' values from the first and second items in the 'yes_symptoms' list.
    """
    try:
        data = eval(summary)  # Evaluate the string as a Python dictionary
        if isinstance(data, dict) and 'yes_symptoms' in data:
            answers = []
            # Process only the first two items if available
            for symptom in data['yes_symptoms'][0:2]:
                if 'answers' in symptom:
                    answers.extend(symptom['answers'])
            return ', '.join(answers)  # Join the answers with commas
        else:
            return None  # Return None if the structure is not as expected
    except (SyntaxError, TypeError, KeyError):
        return None  # Return None if there's an error

In [6]:
def process_dataframe(df, summary_col='summary', symptoms_note_col='symptoms_note'):
    """
    Processes a DataFrame to extract 'answers' and create a new 'symptoms_note' column.
    """
    df[symptoms_note_col] = df[summary_col].apply(extract_answers)
    return df

In [7]:
Symptoms = process_dataframe(data)

In [8]:
Symptoms.head()

,summary,search_term,symptoms_note
0,"{""diseases"": [], ""procedures"": [], ""no_symptom...","มีเสมหะ, ไอ","ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว, ระยะเวลา ไม..."
1,"{""diseases"": [], ""procedures"": [], ""no_symptom...","ไอ, น้ำมูกไหล","ระยะเวลา 1-3 สัปดาห์, ลักษณะ ไอไม่มีเสมหะ ไอแห..."
2,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ปวดท้อง,"บริเวณ รอบๆสะดือ, ระยะเวลา ตั้งแต่ 1 วัน ถึง 1..."
3,"{""diseases"": [], ""procedures"": [], ""no_symptom...",น้ำมูกไหล,"ระยะเวลา น้อยกว่า 10 วัน, ประวัติ ATK ยังไม่ได..."
4,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ตาแห้ง,การรักษาก่อนหน้า ไม่เคย


In [9]:
# Define dictionaries
relevant_keywords = [
    # English
    "cough", "sputum", "phlegm", "pain", "numbness", "weakness", "severity", "location",
    "fever", "headache", "dizziness", "sore throat", "fatigue",
    # Thai
    "ลักษณะ", "ไอ", "เสมหะ", "ปวด", "ชาปลายมือ", "ชาปลายเท้า", "อ่อนแรง", "ไข้", "เวียนหัว", "เจ็บคอ", "เหนื่อย"
]

irrelevant_keywords = [
    # English
    "history", "previous treatment", "atk", "covid", "test", "close contact",
    # Thai
    "ประวัติ", "การรักษาก่อนหน้า", "ระยะเวลา", "ใกล้ชิด"
]

_splitter = re.compile(r"[.;:\-\n]+")

def clean_text(text: str):
    """Return only symptom-relevant chunks from mixed Thai/English notes."""
    if pd.isna(text) or not str(text).strip():
        return pd.NA
    text = str(text)

    chunks = _splitter.split(text)
    kept = []
    seen = set()

    for raw in chunks:
        c = raw.strip()
        if not c:
            continue
        # drop clearly irrelevant chunks
        if any(k in c.lower() for k in (w.lower() for w in irrelevant_keywords)):
            continue
        # keep chunks that mention symptom-relevant concepts
        if any(k in c.lower() for k in (w.lower() for w in relevant_keywords)):
            if c not in seen:
                seen.add(c)
                kept.append(c)

    if not kept:
        return pd.NA
    return " ".join(kept)  # join chunks with spaces

# --- apply to your dataframe ---
Symptoms["symptoms_note_clean"] = Symptoms["symptoms_note"].apply(clean_text)

In [10]:
def clean_text(text: str):
    """Return only symptom-relevant chunks from mixed Thai/English notes."""
    if pd.isna(text) or not str(text).strip():
        return pd.NA
    text = str(text)

    chunks = _splitter.split(text)
    kept = []
    seen = set()

    for raw in chunks:
        c = raw.strip()
        if not c:
            continue
        # drop clearly irrelevant chunks
        if any(k in c.lower() for k in (w.lower() for w in irrelevant_keywords)):
            continue
        # keep chunks that mention symptom-relevant concepts
        if any(k in c.lower() for k in (w.lower() for w in relevant_keywords)):
            if c not in seen:
                seen.add(c)
                kept.append(c)

    if not kept:
        return pd.NA
    return " ".join(kept)  # join chunks with spaces

# --- apply to your dataframe ---
Symptoms["symptoms_note_clean"] = Symptoms["symptoms_note"].apply(clean_text)

In [11]:
Symptoms.head()

,summary,search_term,symptoms_note,symptoms_note_clean
0,"{""diseases"": [], ""procedures"": [], ""no_symptom...","มีเสมหะ, ไอ","ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว, ระยะเวลา ไม...",<NA>
1,"{""diseases"": [], ""procedures"": [], ""no_symptom...","ไอ, น้ำมูกไหล","ระยะเวลา 1-3 สัปดาห์, ลักษณะ ไอไม่มีเสมหะ ไอแห...",<NA>
2,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ปวดท้อง,"บริเวณ รอบๆสะดือ, ระยะเวลา ตั้งแต่ 1 วัน ถึง 1...",<NA>
3,"{""diseases"": [], ""procedures"": [], ""no_symptom...",น้ำมูกไหล,"ระยะเวลา น้อยกว่า 10 วัน, ประวัติ ATK ยังไม่ได...",<NA>
4,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ตาแห้ง,การรักษาก่อนหน้า ไม่เคย,<NA>


In [12]:
Symptoms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   summary              1000 non-null   object
 1   search_term          1000 non-null   object
 2   symptoms_note        1000 non-null   object
 3   symptoms_note_clean  112 non-null    object
dtypes: object(4)
memory usage: 31.4+ KB


In [13]:
data.drop(['summary', 'symptoms_note'], axis=1, inplace=True)

In [14]:
# Keep only rows where 'symptoms_note_clean' is not missing
df_filtered = Symptoms.dropna(subset=['symptoms_note_clean'])

In [15]:
# If you want to ensure type stability (e.g., strings), you can further filter out empty strings after the drop:
df_filtered = df_filtered[df_filtered['symptoms_note_clean'].astype(str).str.strip() != '']

In [16]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112 entries, 12 to 998
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   search_term          112 non-null    object
 1   symptoms_note_clean  112 non-null    object
dtypes: object(2)
memory usage: 2.6+ KB


In [17]:
df_filtered.head()

,search_term,symptoms_note_clean
12,"เสมหะไหลลงคอ, มีเสมหะ",ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว
15,"บวม, ปวดข้อเท้า","ตำแหน่ง บวมที่ข้อ เช่น ข้อเข่า ข้อเท้า, ระดับ ..."
23,"หายใจมีเสียงวี๊ด, หายใจหอบเหนื่อย",ระดับ เหนื่อยเล็กน้อย เหนื่อยง่ายขึ้นเวลาออกแร...
31,"ปวดหัว, ชา",ระดับ ปวดเล็กน้อย ส่งผลต่อการดำเนินกิจวัตรประจ...
44,"ไข้, ไอ","3 วัน, ระดับ ไข้ปานกลาง (38"


In [18]:
table_name = 'df_filtered'
conn = sqlite3.connect('AI_Symptomps_Note.db')  # inline path
df_filtered.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()